# Web scraping hero information
- we will get counter and combo win rate from dota stratz
- we will get laning information from dota stratz too

## get hero info from dota stratz
- get it from https://stratz.com/heroes

In [52]:
import logging
import time
import undetected_chromedriver as uc
import base64

# from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import selenium.common.exceptions as Exceptions
from selenium import webdriver
import math
from tqdm.auto import tqdm
import os
import pickle
import numpy as np

In [53]:
profile_folder = "~/.config/google-chrome"
 	
# load options
options = webdriver.ChromeOptions()
options.add_argument('--user-data-dir=<profile-folder>')
options.add_argument('--profile-directory=Default')
options.page_load_strategy = "normal"
options.headless = False
browser = uc.Chrome(
            version_main=114,
            options=options,
        )


## Obtain all hero urls

In [54]:
HERO_URL = "https://stratz.com/heroes"

In [55]:
# load page 
browser.get(HERO_URL)
time.sleep(2)

In [56]:
heroelexpath = "//a[@id and //ancestor::div[contains(@class, 'hitagi__sc-1q1tiz9-1 gMqHDI')]]"
eles = browser.find_elements(By.XPATH, heroelexpath)

In [57]:
# get all the heros url 
hero_urls = [t.get_attribute('href') for t in eles]
hero_urls[0]

'https://stratz.com/heroes/73'

In [58]:
# now loop each hero url 
cur_url = next(iter(hero_urls))
browser.get(cur_url)

## Obtain laning info

In [59]:
Lane_rate_info_dict = dict()
Url_name_dict = dict()
# dict[heroname] = {["Pos x Pick Rate" : val], "url": val, }

In [60]:
def update_lane_rate_info_for_one_hero(browser, Lane_rate_info_dict, Url_name_dict, cur_url):
    # browser load 
    browser.get(cur_url)
    name_xp = "//span[ contains(@class, 'lbmJJw')]"
    name_ele = browser.find_element(By.XPATH, name_xp)
    hero_name = name_ele.text
    Lane_rate_info_dict[hero_name] = dict()
    Lane_rate_info_dict[hero_name]['url'] = cur_url
    Url_name_dict[cur_url] = hero_name
    # pos 1 2 3 4 5 
    pos_1_lanexp = "//span[contains(text(), 'Safe Lane')]/../../../../div[2]/div[1]/div[1]"
    pos_2_lanexp = "//span[contains(text(), 'Mid Lane')]/../../../../div[2]/div[1]/div[1]"
    pos_3_lanexp = "//span[contains(text(), 'Off Lane')]/../../../../div[2]/div[1]/div[1]"
    pos_4_lanexp = "//span[contains(text(), 'Soft Support')]/../../../../div[2]/div[1]/div[1]"
    pos_5_lanexp = "//span[contains(text(), 'Hard Support')]/../../../../div[2]/div[1]/div[1]"
    lanexps = {"Pos 1 Pick Rate": pos_1_lanexp,
               "Pos 2 Pick Rate": pos_2_lanexp,
               "Pos 3 Pick Rate": pos_3_lanexp,
               "Pos 4 Pick Rate": pos_4_lanexp,
               "Pos 5 Pick Rate": pos_5_lanexp}
    for key, xp in lanexps.items():
        lane_rate_ele = browser.find_element(By.XPATH, xp)
        lane_rate = round(float(lane_rate_ele.text.strip('%')) / 100.0, 3)
        Lane_rate_info_dict[hero_name][key] = lane_rate

In [61]:
update_lane_rate_info_for_one_hero(browser, Lane_rate_info_dict, Url_name_dict, cur_url)

In [62]:
Lane_rate_info_dict

{'Alchemist': {'url': 'https://stratz.com/heroes/73',
  'Pos 1 Pick Rate': 0.293,
  'Pos 2 Pick Rate': 0.274,
  'Pos 3 Pick Rate': 0.224,
  'Pos 4 Pick Rate': 0.117,
  'Pos 5 Pick Rate': 0.092}}

In [63]:
Url_name_dict

{'https://stratz.com/heroes/73': 'Alchemist'}

In [64]:
# wait for the progress
for cur_url in tqdm(hero_urls):
    update_lane_rate_info_for_one_hero(browser, Lane_rate_info_dict,Url_name_dict, cur_url)

  0%|          | 0/124 [00:00<?, ?it/s]

## Obtain winrate info

In [65]:
Hero_versus_with_winrate_dict = dict()
# dict[heroname]["pair_hero_name"] = {versus_win_rate, with_win_rate}

In [69]:
def update_win_rate_info_for_one_hero(browser, Url_name_dict, Hero_versus_with_winrate_dict, cur_url):
    winrate_url = os.path.join(cur_url, "matchups")
    hero_name = Url_name_dict[cur_url]
    Hero_versus_with_winrate_dict[hero_name] = dict()
    # browser load 
    browser.get(winrate_url) 
    eachrowxp = "//a[contains(@class, 'hitagi__sc-1hmgwow-0')]"
    
    test= WebDriverWait(browser, 20).until(EC.visibility_of_element_located((By.XPATH, eachrowxp)))
    time.sleep(5)
    roweles = browser.find_elements(By.XPATH, eachrowxp)
    for tele in roweles:
        pair_url = "/".join(tele.get_attribute('href').split('/')[:-1])
        if pair_url in Url_name_dict:
            pair_hero_name = Url_name_dict[pair_url]
        else:
            raise RuntimeError(f"{pair_url} not found in Url_name_dict")
        Hero_versus_with_winrate_dict[hero_name][pair_hero_name] = dict()
        versus_win_rate_xp = "./div[4]/div[1]"
        with_win_rate_xp = "./div[9]/div[1]"
        counter_rate_xp = "./div[5]/div[1]"
        versus_win_rate_ele = tele.find_element(By.XPATH, versus_win_rate_xp)
        with_win_rate_ele = tele.find_element(By.XPATH, with_win_rate_xp)
        counter_rate_ele = tele.find_element(By.XPATH, counter_rate_xp)
        versus_win_rate = round(float(versus_win_rate_ele.text.strip('%')) / 100.0, 3)
        with_win_rate = round(float(with_win_rate_ele.text.strip('%')) / 100.0, 3)
        counter_rate_num_t = counter_rate_ele.text.strip('%')
        if  counter_rate_num_t[0] == "~":
            counter_rate_num_t = counter_rate_num_t[1:]
        counter_rate = round(float(counter_rate_num_t) / 100.0, 3)
        Hero_versus_with_winrate_dict[hero_name][pair_hero_name]['Versus Win Rate'] = versus_win_rate
        Hero_versus_with_winrate_dict[hero_name][pair_hero_name]['Match Win Rate'] = with_win_rate
        Hero_versus_with_winrate_dict[hero_name][pair_hero_name]['Counter Rate'] = counter_rate

In [72]:
# wait for the progress
for ind, cur_url in enumerate(tqdm(hero_urls)):
    # if ind < 113:
    #     continue
    heroname = Url_name_dict[cur_url]
    update_win_rate_info_for_one_hero(browser, Url_name_dict, Hero_versus_with_winrate_dict, cur_url)


  0%|          | 0/124 [00:00<?, ?it/s]

In [74]:
# wait for the progress
for ind, cur_url in enumerate(tqdm(hero_urls)):
    heroname = Url_name_dict[cur_url]
    if len(Hero_versus_with_winrate_dict[heroname]) == 123: # fill up incomplete data
        continue 
    update_win_rate_info_for_one_hero(browser, Url_name_dict, Hero_versus_with_winrate_dict, cur_url)

  0%|          | 0/124 [00:00<?, ?it/s]

In [85]:
browser.close()

## Save the intermediate information dict

In [75]:
count = 0
for name, valdic in Hero_versus_with_winrate_dict.items():
    if len(valdic) != 123:
        print(name)
        # print(valdic)
        count += 1
print(f"find {count} incomplete data")

find 0 incomplete data


In [ ]:
# # we need to store three dict
# Hero_versus_with_winrate_dict
# Lane_rate_info_dict
# Url_name_dict

In [76]:
with open("../records/hero_versus_with_winrate_dict.pkl", 'wb') as f:
    pickle.dump(Hero_versus_with_winrate_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

with open("../records/lane_rate_info_dict.pkl", 'wb') as f:
    pickle.dump(Lane_rate_info_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

with open("../records/url_name_dict.pkl", 'wb') as f:
    pickle.dump(Url_name_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

# Form 2d matrix
- we need 2 2d matrices, one is for with winrate and the other one is for counter winrate
- with winrate matrix should be symetrical
- counter winrate matrix should be complement

`dict[ours][opponent] = winrate`

In [77]:
with open("../records/hero_versus_with_winrate_dict.pkl", 'rb') as f:
    Hero_versus_with_winrate_dict= pickle.load(f)

with open("../records/lane_rate_info_dict.pkl", 'rb') as f:
    Lane_rate_info_dict = pickle.load(f)

with open("../records/url_name_dict.pkl", 'rb') as f:
    Url_name_dict = pickle.load(f)

In [78]:
# with winrate matrix 
with_winrate_matrix = dict()
for heroname in tqdm(Hero_versus_with_winrate_dict.keys()):
    with_winrate_matrix[heroname] = dict()
    for paired_heroname in Hero_versus_with_winrate_dict[heroname].keys():
        with_winrate_matrix[heroname][paired_heroname] = Hero_versus_with_winrate_dict[heroname][paired_heroname]['Match Win Rate']

  0%|          | 0/124 [00:00<?, ?it/s]

In [79]:
# with winrate matrix force symmetricity 
with_winrate_matrix_symmetricity = dict()
for heroname in tqdm(with_winrate_matrix.keys()):
    with_winrate_matrix_symmetricity[heroname] = dict()
    for pairheroname in with_winrate_matrix[heroname].keys():
        fwdwinrate = with_winrate_matrix[heroname][pairheroname]
        bwdwinrate = with_winrate_matrix[pairheroname][heroname]
        symwinrate = round((fwdwinrate + bwdwinrate) / 2.0, 3)
        with_winrate_matrix_symmetricity[heroname][pairheroname] = symwinrate

  0%|          | 0/124 [00:00<?, ?it/s]

In [80]:
# versus winrate matrix 
versus_winrate_matrix = dict()
for heroname in tqdm(Hero_versus_with_winrate_dict.keys()):
    versus_winrate_matrix[heroname] = dict()
    for paired_heroname in Hero_versus_with_winrate_dict[heroname].keys():
        versus_winrate_matrix[heroname][paired_heroname] = Hero_versus_with_winrate_dict[heroname][paired_heroname]['Versus Win Rate']

  0%|          | 0/124 [00:00<?, ?it/s]

In [81]:
# versus winrate matrix force complement 
versus_winrate_matrix_complement = dict()
for heroname in tqdm(versus_winrate_matrix.keys()):
    versus_winrate_matrix_complement[heroname] = dict()
    for opponentheroname in versus_winrate_matrix[heroname].keys():
        yourwinrate = versus_winrate_matrix[heroname][opponentheroname]
        enemywinrate = versus_winrate_matrix[opponentheroname][heroname]
        yourwinratebyenemy = 1.0 - enemywinrate
        yourwinrate_norm = round((yourwinrate + yourwinratebyenemy)/2.0, 3)
        enemywinrate_norm = 1.0 - yourwinrate_norm
        versus_winrate_matrix_complement[heroname][opponentheroname] = yourwinrate_norm

  0%|          | 0/124 [00:00<?, ?it/s]

In [82]:
# evaluate
print(with_winrate_matrix_symmetricity['Huskar']['Dazzle'])
print(with_winrate_matrix_symmetricity['Dazzle']['Huskar'])
print(versus_winrate_matrix_complement['Meepo']['Elder Titan'])
print(versus_winrate_matrix_complement['Elder Titan']['Meepo'])

0.522
0.522
0.444
0.556


In [83]:
# counter rate matrix
counter_rate_matrix = dict()
for heroname in tqdm(Hero_versus_with_winrate_dict.keys()):
    counter_rate_matrix[heroname] = dict()
    for paired_heroname in Hero_versus_with_winrate_dict[heroname].keys():
        counter_rate_matrix[heroname][paired_heroname] = Hero_versus_with_winrate_dict[heroname][paired_heroname]['Counter Rate']

  0%|          | 0/124 [00:00<?, ?it/s]

In [84]:
with open("../records/with_winrate_matrix.pkl", 'wb') as f:
    pickle.dump(with_winrate_matrix_symmetricity, f, protocol=pickle.HIGHEST_PROTOCOL)

with open("../records/versus_winrate_matrix.pkl", 'wb') as f:
    pickle.dump(versus_winrate_matrix_complement, f, protocol=pickle.HIGHEST_PROTOCOL)

with open("../records/counter_rate_matrix.pkl", 'wb') as f:
    pickle.dump(counter_rate_matrix, f, protocol=pickle.HIGHEST_PROTOCOL)

# Get default Pos 1-5 hero pool
- we use the lane pick rate info to get this default pos 1-5 hero pool
- a hero can only have at most 2 positions
- manually edit it here

In [27]:
Pos_1_list = [] 
Pos_2_list = [] 
Pos_3_list = [] 
Pos_4_list = [] 
Pos_5_list = [] 

for heroname in tqdm(Lane_rate_info_dict):
    pickrates = [
                    Lane_rate_info_dict[heroname]['Pos 1 Pick Rate'],
                    Lane_rate_info_dict[heroname]['Pos 2 Pick Rate'],
                    Lane_rate_info_dict[heroname]['Pos 3 Pick Rate'],
                    Lane_rate_info_dict[heroname]['Pos 4 Pick Rate'],
                    Lane_rate_info_dict[heroname]['Pos 5 Pick Rate'],
                ]
    max_inds = np.argsort(pickrates)[::-1][:2]
    if 0 in max_inds:
        Pos_1_list.append(heroname)
    if 1 in max_inds:
        Pos_2_list.append(heroname)
    if 2 in max_inds:
        Pos_3_list.append(heroname)
    if 3 in max_inds:
        Pos_4_list.append(heroname)
    if 4 in max_inds:
        Pos_5_list.append(heroname)

  0%|          | 0/124 [00:00<?, ?it/s]

In [31]:
# evaluate
print('Meepo' in Pos_1_list)
print('Meepo' in Pos_2_list)
print('Meepo' in Pos_3_list)
print('Shadow Demon' in Pos_3_list)
print('Shadow Demon' in Pos_5_list)
print('Shadow Demon' in Pos_4_list)

True
True
False
False
True
True


In [50]:
# get size for each pos hero pool
print(len(Pos_1_list))
print(len(Pos_2_list))
print(len(Pos_3_list))
print(len(Pos_4_list))
print(len(Pos_5_list))

30
34
34
47
33


## Manually remove some hero in default e.g., meepo is too hard for normal players

In [39]:
Pos_1_list

['Alchemist',
 'Chaos Knight',
 'Lifestealer',
 'Sven',
 'Wraith King',
 'Anti-Mage',
 'Arc Warden',
 'Bloodseeker',
 'Drow Ranger',
 'Faceless Void',
 'Gyrocopter',
 'Juggernaut',
 'Lone Druid',
 'Luna',
 'Medusa',
 'Monkey King',
 'Morphling',
 'Naga Siren',
 'Phantom Assassin',
 'Phantom Lancer',
 'Riki',
 'Slark',
 'Sniper',
 'Spectre',
 'Templar Assassin',
 'Terrorblade',
 'Troll Warlord',
 'Ursa',
 'Weaver',
 'Muerta']

In [34]:
Unwanted_Pos_1_list = ['Bristleback','Ember Spirit','Meepo','Shadow Fiend']
for unwanted_hero in Unwanted_Pos_1_list:
    Pos_1_list.remove(unwanted_hero)

In [38]:
Pos_1_list.insert(12, 'Lone Druid')

In [46]:
Pos_2_list

['Alchemist',
 'Dragon Knight',
 'Earth Spirit',
 'Huskar',
 'Kunkka',
 'Primal Beast',
 'Anti-Mage',
 'Clinkz',
 'Ember Spirit',
 'Medusa',
 'Pugna',
 'Monkey King',
 'Morphling',
 'Razor',
 'Riki',
 'Shadow Fiend',
 'Sniper',
 'Templar Assassin',
 'Viper',
 'Death Prophet',
 'Leshrac',
 'Lina',
 'Necrophos',
 'Outworld Destroyer',
 'Puck',
 'Queen of Pain',
 'Storm Spirit',
 'Tinker',
 'Zeus',
 'Invoker',
 'Lone Druid',
 'Pangolier',
 'Void Spirit',
 'Windranger']

In [40]:
Unwanted_Pos_2_list = ['Legion Commander', 'Mars', 'Night Stalker','Slardar','Arc Warden', 'Bloodseeker', 'Drow Ranger', 
                       'Luna', 'Meepo', 'Phantom Assassin', 'Slark', 'Weaver', 'Muerta', 'Batrider', 'Brewmaster', 'Broodmother',
                       'Lycan', 'Timbersaw', 'Visage'
                      ]
for unwanted_hero in Unwanted_Pos_2_list:
    Pos_2_list.remove(unwanted_hero)

In [45]:
Pos_2_list.insert(10, 'Pugna')

In [42]:
Pos_3_list

['Axe',
 'Bristleback',
 'Centaur Warrunner',
 'Chaos Knight',
 'Dawnbreaker',
 'Doom',
 'Dragon Knight',
 'Earthshaker',
 'Elder Titan',
 'Huskar',
 'Kunkka',
 'Legion Commander',
 'Lifestealer',
 'Mars',
 'Night Stalker',
 'Omniknight',
 'Primal Beast',
 'Slardar',
 'Spirit Breaker',
 'Sven',
 'Tidehunter',
 'Tiny',
 'Tusk',
 'Underlord',
 'Wraith King',
 'Faceless Void',
 'Juggernaut',
 'Naga Siren',
 'Phantom Lancer',
 'Razor',
 'Spectre',
 'Terrorblade',
 'Troll Warlord',
 'Ursa',
 'Viper',
 'Death Prophet',
 'Necrophos',
 'Outworld Destroyer',
 'Abaddon',
 'Beastmaster',
 'Brewmaster',
 'Broodmother',
 'Dark Seer',
 'Lone Druid',
 'Lycan',
 'Magnus',
 'Pangolier',
 'Sand King',
 'Timbersaw',
 'Visage',
 'Void Spirit']

In [43]:
Unwanted_Pos_3_list = ['Elder Titan', 'Lifestealer', 'Sven', 'Tiny', 'Wraith King', 'Faceless Void', 'Juggernaut', 'Naga Siren', 'Phantom Lancer', 'Spectre',
                       'Terrorblade','Troll Warlord','Ursa','Broodmother','Lone Druid','Visage','Void Spirit'
                      ]
for unwanted_hero in Unwanted_Pos_3_list:
    Pos_3_list.remove(unwanted_hero)

In [44]:
Pos_4_list

['Axe',
 'Centaur Warrunner',
 'Dawnbreaker',
 'Doom',
 'Earth Spirit',
 'Earthshaker',
 'Elder Titan',
 'Ogre Magi',
 'Omniknight',
 'Pudge',
 'Spirit Breaker',
 'Tidehunter',
 'Tiny',
 'Treant Protector',
 'Tusk',
 'Underlord',
 'Undying',
 'Bounty Hunter',
 'Clinkz',
 'Gyrocopter',
 'Hoodwink',
 'Ancient Apparition',
 'Crystal Maiden',
 'Disruptor',
 'Enchantress',
 'Grimstroke',
 'Jakiro',
 'Keeper of the Light',
 'Leshrac',
 'Lich',
 'Lina',
 'Lion',
 "Nature's Prophet",
 'Oracle',
 'Puck',
 'Pugna',
 'Queen of Pain',
 'Rubick',
 'Shadow Demon',
 'Shadow Shaman',
 'Silencer',
 'Skywrath Mage',
 'Storm Spirit',
 'Tinker',
 'Warlock',
 'Witch Doctor',
 'Zeus',
 'Bane',
 'Batrider',
 'Beastmaster',
 'Chen',
 'Clockwerk',
 'Dark Seer',
 'Dark Willow',
 'Dazzle',
 'Enigma',
 'Invoker',
 'Io',
 'Magnus',
 'Marci',
 'Mirana',
 'Nyx Assassin',
 'Phoenix',
 'Sand King',
 'Snapfire',
 'Techies',
 'Vengeful Spirit',
 'Venomancer',
 'Windranger',
 'Winter Wyvern']

In [47]:
Unwanted_Pos_4_list = ['Axe','Centaur Warrunner','Dawnbreaker','Earthshaker','Tidehunter','Underlord','Clinkz','Gyrocopter','Leshrac','Lina','Puck',
                       'Queen of Pain','Storm Spirit','Tinker','Zeus','Beastmaster','Chen','Dark Seer', 'Enigma', 'Invoker', 'Magnus',
                       'Sand King','Windranger'
                      ]
for unwanted_hero in Unwanted_Pos_4_list:
    Pos_4_list.remove(unwanted_hero)

In [48]:
Pos_5_list

['Ogre Magi',
 'Pudge',
 'Treant Protector',
 'Undying',
 'Bounty Hunter',
 'Hoodwink',
 'Ancient Apparition',
 'Crystal Maiden',
 'Disruptor',
 'Enchantress',
 'Grimstroke',
 'Jakiro',
 'Keeper of the Light',
 'Lich',
 'Lion',
 "Nature's Prophet",
 'Oracle',
 'Pugna',
 'Rubick',
 'Shadow Demon',
 'Shadow Shaman',
 'Silencer',
 'Skywrath Mage',
 'Warlock',
 'Witch Doctor',
 'Abaddon',
 'Bane',
 'Chen',
 'Clockwerk',
 'Dark Willow',
 'Dazzle',
 'Enigma',
 'Io',
 'Marci',
 'Mirana',
 'Nyx Assassin',
 'Phoenix',
 'Snapfire',
 'Techies',
 'Vengeful Spirit',
 'Venomancer',
 'Winter Wyvern']

In [49]:
Unwanted_Pos_5_list = ['Pudge',"Nature's Prophet",'Chen','Enigma','Marci','Mirana','Nyx Assassin','Snapfire','Venomancer'
                      ]
for unwanted_hero in Unwanted_Pos_5_list:
    Pos_5_list.remove(unwanted_hero)

In [51]:
# save these default position hero pools 
with open("../records/default_pos_1_hero_pool.txt", 'w') as f:
    f.write(str(Pos_1_list))

with open("../records/default_pos_2_hero_pool.txt", 'w') as f:
    f.write(str(Pos_2_list))

with open("../records/default_pos_3_hero_pool.txt", 'w') as f:
    f.write(str(Pos_3_list))

with open("../records/default_pos_4_hero_pool.txt", 'w') as f:
    f.write(str(Pos_4_list))

with open("../records/default_pos_5_hero_pool.txt", 'w') as f:
    f.write(str(Pos_5_list))